In [33]:
# RS- Word2Vec new Embedding- 

In [35]:
# Pyncone

In [34]:
## Large- Error with embeddings
# ----------------------------
# Word2Vec - > Embeddings - Initialized - new article ? -> How to create a vector for this new

## Context based Approach using Tf-idf, Word2Vec

In [16]:
# Importing the necessary libraries
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec
import numpy as np

import subprocess
subprocess.run(["python", "-m", "spacy", "download", "da_core_news_sm"])

nlp = spacy.load('da_core_news_sm')

# Download Danish stop words from NLTK
nltk.download('stopwords')
danish_stop_words = set(stopwords.words('danish'))
# Load the Danish spaCy model
spacy.cli.download("da_core_news_sm")
nlp = spacy.load('da_core_news_sm')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giwrg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('da_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## TF-idf

#### Import the dataset

In [22]:
# Load the dataset
size = 'demo'
type_ = 'train'
articles_path = f"./files/parquet/ebnerd_{size}/articles.parquet"
df_articles = pd.read_parquet(articles_path)

# Work only with the body column
df_articles_body = df_articles[['body']]

#### Preprocess the text

In [23]:
def preprocess_text(text):
    text = re.sub(r'[^a-zA-ZæøåÆØÅ\s]', '', text)
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if token.text not in danish_stop_words and not token.is_punct]
    return ' '.join(tokens)

# Apply preprocessing to the dataset
df_articles_body['processed_content'] = df_articles_body['body'].apply(preprocess_text)
df_articles_body.drop("body", axis=1, inplace=True)

C:\Users\giwrg\AppData\Local\Temp\ipykernel_10448\238926766.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_articles_body['processed_content'] = df_articles_body['body'].apply(preprocess_text)
C:\Users\giwrg\AppData\Local\Temp\ipykernel_10448\238926766.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_articles_body.drop("body", axis=1, inplace=True)


#### Cosine Similarity

In [24]:
# Load the Tf-idf model and apply it to the docs
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_articles_body['processed_content'])

# Find the cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim_df = pd.DataFrame(cosine_sim, index=df_articles_body.index, columns=df_articles_body.index)

#### Recommendation

In [25]:
# Example function to get similar articles
def get_similar_articles(article_index, cosine_sim_matrix, top_n=5):
    sim_scores = list(enumerate(cosine_sim_matrix[article_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    similar_articles = [(index, score) for index, score in sim_scores]
    return similar_articles

# Example usage
article_index = 0 
similar_articles = get_similar_articles(article_index, cosine_sim)
print(f"Articles similar to article {article_index}: {similar_articles}")

Articles similar to article 0: [(3430, 0.22637903660754571), (4255, 0.20733735683736004), (9268, 0.18233629146389696), (792, 0.17701128174711836), (2626, 0.1584519494329333)]


## Word2Vec

#### Import the dataset

In [26]:
# Load the dataset once again 
df_articles_body = df_articles[['body']]

#### Preprocess the text

In [27]:
def preprocess_text_w2v(text):
    text = re.sub(r'[^a-zA-ZæøåÆØÅ\s]', '', text)
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if token.text not in danish_stop_words and not token.is_punct]
    return tokens

# Apply preprocessing to the dataset
df_articles_body['processed_content'] = df_articles_body['body'].apply(preprocess_text_w2v)


C:\Users\giwrg\AppData\Local\Temp\ipykernel_10448\3570535638.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_articles_body['processed_content'] = df_articles_body['body'].apply(preprocess_text_w2v)


#### Train Word2Vec model

In [28]:
sentences = df_articles_body['processed_content'].tolist()
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

#### Function to compute the average vector for a document

In [29]:
def document_vector(doc, model):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in model.wv.index_to_key]
    if len(doc) == 0:
        return np.zeros(model.vector_size)
    return np.mean(model.wv[doc], axis=0)

#### Compute document vectors

In [30]:
df_articles_body['doc_vector'] = df_articles_body['processed_content'].apply(lambda x: document_vector(x, word2vec_model))
doc_vectors = np.stack(df_articles_body['doc_vector'].values)

C:\Users\giwrg\AppData\Local\Temp\ipykernel_10448\3058639058.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_articles_body['doc_vector'] = df_articles_body['processed_content'].apply(lambda x: document_vector(x, word2vec_model))


#### Cosine similarity

In [31]:
cosine_sim = cosine_similarity(doc_vectors)
cosine_sim_df = pd.DataFrame(cosine_sim, index=df_articles_body.index, columns=df_articles_body.index)

In [32]:
def get_similar_articles(article_index, cosine_sim_matrix, df, top_n=5):
    sim_scores = list(enumerate(cosine_sim_matrix[article_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    similar_articles = [(df.index[i], score) for i, score in sim_scores]
    return similar_articles
# Example 
article_index = 0 
similar_articles = get_similar_articles(article_index, cosine_sim, df_articles_body)
print("Similar Articles:")
print(similar_articles)

Similar Articles:
[(439, 0.98844374983732), (9304, 0.9878260624412174), (2975, 0.9871099786582396), (9041, 0.9870656407868641), (2427, 0.9868525744860396)]
